# Імпортування даних у таблиці вимірів сховища даних

Імпортуємо бібліотеки

In [99]:
import pandas as pd
from dotenv import dotenv_values
from sqlalchemy import create_engine, text
from sqlalchemy.sql import select

Завантажуємо змінні оточення з .env файлу для з'єднання зі сховищем

In [64]:
config = dotenv_values()

DB_NAME = config.get('DB_NAME')
DB_USER = config.get('DB_USER')
DB_HOST = config.get('DB_HOST')
DB_PASSWORD = config.get('DB_PASSWORD')
DB_PORT = config.get('DB_PORT')

З'єднуємося з базою даних

In [75]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

## Countries Table

Імпортуємо дані з датасету

In [70]:
countries_df = pd.read_csv('../data/f1db-countries.csv')
countries_df.head()

,id,alpha2Code,alpha3Code,name,demonym,continentId
0,afghanistan,AF,AFG,Afghanistan,Afghan,asia
1,aland-islands,AX,ALA,Åland Islands,Ålandic,europe
2,albania,AL,ALB,Albania,Albanian,europe
3,algeria,DZ,DZA,Algeria,Algerian,africa
4,american-samoa,AS,ASM,American Samoa,Samoan,australia


Приводимо дані у потрібний нам формат для сховища

In [71]:
countries_df = countries_df[['alpha3Code', 'name', 'demonym', 'id']]
countries_df.head()

,alpha3Code,name,demonym,id
0,AFG,Afghanistan,Afghan,afghanistan
1,ALA,Åland Islands,Ålandic,aland-islands
2,ALB,Albania,Albanian,albania
3,DZA,Algeria,Algerian,algeria
4,ASM,American Samoa,Samoan,american-samoa


Перейменуємо стовпці

In [74]:
countries_df.columns = ['alpha3_code', 'name', 'demonym', 'id']
countries_df.head()

,alpha3_code,name,demonym,id
0,AFG,Afghanistan,Afghan,afghanistan
1,ALA,Åland Islands,Ålandic,aland-islands
2,ALB,Albania,Albanian,albania
3,DZA,Algeria,Algerian,algeria
4,ASM,American Samoa,Samoan,american-samoa


Створимо функцію, яка внесе потрібні нам дані у сховище та поверне згенеровані ідентифікатори для кожного рядка

In [122]:
def insert_data(df, table):
    df.to_sql('countries', engine, if_exists='append', index=False)

    with engine.connect() as connection:
        result = connection.execute(text(f"SELECT id FROM {table}"))
        generated_ids = [row[0] for row in result]
        
    return generated_ids

Вносимо дані з датафрейму у таблицю країн, функція повертає датафрейм з ідентифікаторами, які сховище присвоїло рядкам. Ці ідентифікатори додаємо до датафрейму для того, аби в наступних датафреймах замінити їх.

In [123]:
countries_df['dbId'] = insert_data(countries_df[['alpha3_code', 'name', 'demonym']], 'countries')
countries_df.head()

,alpha3_code,name,demonym,id,dbId
0,AFG,Afghanistan,Afghan,afghanistan,1
1,ALA,Åland Islands,Ålandic,aland-islands,2
2,ALB,Albania,Albanian,albania,3
3,DZA,Algeria,Algerian,algeria,4
4,ASM,American Samoa,Samoan,american-samoa,5


## Circuits Table

Імпортуємо дані з датасету

## Constructors Table

## Drivers Table

## Grand Prix Table

## Races Table

## Seasons Table